# Estrazione Nomi

## Da pdf o txt

In [2]:
def join_cognomi_articles(words):
    articles = set([u'de', u'di', u'du', u'del', u'dell', u'la', u'dei', u'le', 
                    u'della', u'dall', u'dalla', u'dello', 'degli', 'lo', 'art',
                   u'dal', u'dalle'])
    found = False
    joined_words = []
    for word in words:
        if not found and word.lower() in articles:
            found = True
            cur_word = word
        elif found:
            cur_word += ' '+word
            joined_words.append(cur_word)
            found = False
        else:
            joined_words.append(word)
    return joined_words

In [3]:
#nomi = list(n.strip().lower() for n in open('../extraction/nomi/nomi_asti.txt'))
#nomi = set(n.strip().lower() for n in open('../extraction/nomi/nomi_gnorry.txt'))
#nomi = set(n.strip().lower() for n in open('../extraction/nomi/nomi_italiani.txt'))
#notais = set(n.strip().lower() for n in open('../extraction/nomi/notari.txt'))
#cognomi = set(n.strip().lower() for n in open('../extraction/nomi/cognomi.txt'))
#cognomi = set(n.strip().lower() for n in open('../extraction/nomi/cognomi_big.txt'))
#nomi_cognomi_list = [w for n in open('../extraction/nomi/notari.txt') for w in n.strip().lower().split(' ')]
nomi_cognomi_list = [w for n in open('../dictionaries/nomi_notai.txt') for w in n.strip().lower().split(' ')]
nomi_cognomi_notai = set(join_cognomi_articles(nomi_cognomi_list))

In [ ]:
nomi_cognomi_notai

In [ ]:
def extract_notaio_from_doc(dirr, is_pdf=False, max=-1):
    text = ""
    for i,f in enumerate(os.listdir(dirr)):
        if i == max:
            break
        if is_pdf and f[-3:] == 'pdf':
            text = textract.process(dirr + f).strip()
        else: 
            with open (dirr + f, 'r') as f2:
                text = f2.read()
        text = wd.replace_evil_dots_and_underscores(text, rep=r' ').replace('\n',' ').replace(',','')
        text = re.sub('\s+', ' ', text).strip()
        found = False
        for i,line in enumerate(wd.split_sents(text)):
            if found:
                break
            l = line.strip()
            if l != '':
                words = l.split(' ')
                for word in words:
                    if word.lower() == 'me' or word.lower() == 'sottoscritto':
                        index = words.index(word)
                        print(f, ' '.join(words[index+1:index+6]))
                        found = True
                        break
        if not found:
            print('NON TROVATO', '\t\t', f)

In [ ]:
extract_notaio_from_doc('../extraction/files_to_label/', True)

In [ ]:
extract_notaio_from_doc('../atti_costitutivi_txt/', False, 500)

## Dalle frasi

In [ ]:
def extract_notaio_from_sentences(doc):
    '''
    Estrai notaio da un documento.
    Arguments:
        - doc: il documento (lista di frasi)
    Returns:
        - lista di nomi, lista di cognomi
    '''
    #w.lower() in nomi or w.lower() in cognomi or w.lower() in whitelist \or
    whitelist = ['de', 'di'] #, 'dei', 'del', 'della', 'degli', 'delle', 'da', 'dello']
    
    for frase in doc:
        frase = frase.replace('\n',' ').replace(',','')
        frase = re.sub('\s+', ' ', frase).strip()
        if frase != '':
            words = re.split('\W', frase)
            #frase.split(' ')
            for word in words:
                if word.lower() == 'me' or word.lower() == 'sottoscritto' or word.lower() == 'mir':
                    index = words.index(word)
                    sub = words[index+1:index+6]
                    res = ''
                    for w in sub:
                        if  any((w.lower() in set(x.lower() for x in riga.split(' '))) for riga in notais):
                            res += w + ' '
                        #if any((w.lower() in set(x.lower() for x in riga.split(' '))) for riga in notais):
                        #    res += w + ' '
                    res +=  '\t\t' +' '.join(sub)   
                    return res     
                    
    return ""

In [ ]:
for i,f in enumerate(os.listdir('../atti_costitutivi_txt/')):
    if i == 500:
        break
    text = ""    
    with open ('../atti_costitutivi_txt/' + f, 'r') as f2:
        text = f2.read()
    sents = wd.sentences_doc(text, rep=r' ')    
    res = extract_notaio_from_sentences(sents)
    print(i, f, '\t', res)    

##### Paolo

In [ ]:
#paolo

def join_apostrophe(words):
    joined = ' '.join(words)
    rep_d = joined.replace(' d ', ' d\'').replace(' D ', ' D\'')
    return rep_d.split()

def extract_notaio_sent(sentence, notaio_names):
    words = join_apostrophe(wd.splitted_words_utf8(sentence))
    words = join_cognomi_articles(words)
    return [word for word in words if word.lower() in notaio_names and word[0].isupper()]

def find_me(words):
    lwords = [w.lower() for w in words]
    im = -1
    arts = set(['me', 'sottoscritto', 'mir'])
    for i, word in enumerate(lwords):
        if any((w in arts) for w in word.split()):
            return i
    return im

def extract_notaio_name(doc, notaio_names, neigh=5, max_names=3):
    words = [w for sent in doc for w in wd.splitted_words_utf8(sent)]
    j_words = join_cognomi_articles(join_apostrophe(words))
    im = find_me(j_words)
    if im >= 0:
        m_words = j_words[im+1:im+neigh]
    else:
        m_words = j_words
    found_names = [word for word in m_words if word.lower() in notaio_names and word[0].isupper()]
    return uniq_list(found_names)[:max_names]
    

In [ ]:
for f in glob.glob('../extraction/esempi_descrizioni2/*.pdf'):
    txt = te.extract_text(f)
    sents = wd.sentences_doc(txt, rep=' ')
    name = extract_notaio_name(sents, nomi_cognomi_notai)
    print(f.split('/')[-1], name)

In [ ]:
txt_dir = '../atti_costitutivi_txt/'
l = []
for i, f in enumerate(os.listdir(txt_dir)):
    txt = open(txt_dir+f).read()
    sents = wd.sentences_doc(txt, rep=' ')
    name = extract_notaio_name(sents, nomi_cognomi_notai)
    l.append(name)
    #print(i)
    #print(f, ' '.join(name))
    #print(txt)


In [ ]:
100*sum(1 for n in l if len(n)>1)/len(l)

In [ ]:
len(os.listdir(txt_dir))

In [ ]:
join_cognomi_articles(u'ciao della maggio uno del pietro caio'.split())

# Estrazione STATUTO

In [1]:
from __future__ import print_function
from __future__ import division
import os
import re
import codecs
import textract
import lib.words as wd
import lib.text_extraction as te
import glob
import lib.article_numbers as an
from shutil import copyfile
from lib.utils import uniq_list

In [ ]:
# Sposta tutti i file contenenti uno statuto nella cartella '../statuti'

def copy_files_with_statuto(dirr, is_pdf=False, max=-1):
    begin_statuto = ['S T A T U T O', 'STATUTO', 'STATUTO DELLA SOCITET', 'FUNZIONAMENTO DELLA SOCIET', 'PATTI DISCIPLINANTI']
    
    for i,f in enumerate(os.listdir(dirr)):
        text = ""
        if i == 0:
            continue
        if i == max:
            break
        if is_pdf and f[-3:] == 'pdf':
            try:
                text = textract.process(dirr + f).strip()
            except:
                continue
        sents = wd.sentences_doc(text, rep=r' ')    
        to_move = False
        
        for k, frase in enumerate(sents):
            if frase.strip() != "":
                for b in begin_statuto:
                    try:
                        j = wd.index_ignore_whitespaces(frase, b)
                    except:
                        print("Exception in: "+frase)
                        j = -1
                    if j > -1:
                        inizio = k
                        index = j
                        to_move = True
                        break
                    
        if to_move:            
            copyfile(str(dirr + f),str("../statuti/" + f))
            
        print(i, f, to_move)        

In [ ]:
copy_files_with_statuto("../atti_costitutivi/", True)

In [12]:
def replace_evil_dots(txt, rep=r''):
    """
    Eliminates dots in abbreviations and numbers, in order to split sentences on dots (that really divide sentences).
    """
    no_abbr = re.sub(u'([bcdfghjklmnpqrstvwxyz])\.(?!\n)', r'\1'+rep, txt)
    no_nums = re.sub(u'(\d)\.(?!\n)', r'\1'+rep, no_abbr)
    no_nums = re.sub(u'\.(\d)', rep + r'\1', no_nums)
    no_maiusc = re.sub(u'([A-Z])\.(?!\n)', r'\1'+rep, no_nums)
    return no_maiusc

def sentences_doc(doc, rep=r''):
    txt = wd.clean_string_not_compressed(doc)
    clean_txt = replace_evil_dots(txt, rep=rep)
    temp = wd.split_sents_newline(clean_txt) 
    res = [x.replace("<DOT>",".") for x in temp]
    #res = [wd.compress_blanks(x.replace("<DOT>",".")) for x in temp]
    return res

In [11]:
def replace_evil_dots_statuto(txt, rep=r''):
    """
    Eliminates dots in abbreviations and numbers, in order to split sentences on dots (that really divide sentences).
    """
    no_abbr = re.sub(u'([bcdfghjklmnpqrstvwxyz])\.(?!\n)', r'\1'+rep, txt)
    no_nums = re.sub(u'(\d)\.(?!\n)', r'\1'+rep, no_abbr)
    no_nums = re.sub(u'\.(\d)', rep + r'\1', no_nums)
    no_maiusc = re.sub(u'([A-Z])\.(?!\n)', r'\1'+rep, no_nums)
    return no_maiusc

def sentences_doc_statuto(doc, rep=r''):
    txt = wd.clean_string_not_compressed(doc)
    clean_txt = replace_evil_dots_statuto(txt, rep=rep)
    temp = wd.split_sents_newline(clean_txt) 
    res = [x.replace("<DOT>",".") for x in temp]
    #res = [wd.compress_blanks(x.replace("<DOT>",".")) for x in temp]
    return res

# Funzione di production, da copiare in article_numbers.py
# Prende in ingresso il testo passato solo da Textract e restituisce una stringa con tutto lo statuto
def extract_statuto(text):
    begin_statuto = ['S T A T U T O', 'STATUTO', 'STATUTO DELLA SOCITET', 'FUNZIONAMENTO DELLA SOCIET', 'PATTI DISCIPLINANTI']
    its_a_trap = ["ATTO COSTITUTIVO", "COSTITUZIONE DI SOCIET", "si allega", "lettera"]
    
    inizio = -1
    index = -1
    fine = -1
    
    # Just to be sure
    text = text.strip()

    sents = sentences_doc_statuto(text, rep=r'<DOT>')  

    for k, frase in enumerate(sents):
        found = False
        if index == -1:
            for t in its_a_trap:
                if wd.index_ignore_whitespaces(frase, t) > -1:
                    found = True
                    break
            if found:
                continue

            for b in begin_statuto:
                j = wd.index_ignore_whitespaces(frase, b)
                if j > -1:
                    inizio = k
                    index = j
                    break

    if index > -1:     
        # end_statuto_text restituisce la prima frase dopo la fine dello statuto
        res = an.end_statuto_init_text(wd.clean_string_not_compressed(text), sents[inizio][index:])
        if res != '':
            res = res + '\n'
            for y, frase2 in reversed(list(enumerate(sents))):
                if res in frase2:
                    fine = y
                    index_fine = frase2.index(res)
                    break

        for z, fr in enumerate(sents):
            if "RELAZIONE DI STIMA" in fr:
                fine = z
                index_fine = fr.index("RELAZIONE DI STIMA")
                break

    if index > -1 and fine > -1:
        to_write = [wd.compress_blanks(s.strip()) for s in sents[inizio+1:fine]]
        return(wd.compress_blanks(sents[inizio][index:]) + to_write + wd.compress_blanks(sents[fine][:index_fine]))
    elif index > -1 and fine == -1:
        to_write = [wd.compress_blanks(s.strip()) for s in sents[inizio+1:]]
        return(wd.compress_blanks(sents[inizio][index:]) + to_write)
    else:
        # Statuto non trovato
        return []

In [5]:
def extract_statuto_from_sentences(dirr, outf, is_pdf=False, max=-1):
    begin_statuto = ['S T A T U T O', 'STATUTO', 'STATUTO DELLA SOCITET', 'FUNZIONAMENTO DELLA SOCIET', 'PATTI DISCIPLINANTI']
    its_a_trap = ["ATTO COSTITUTIVO", "COSTITUZIONE DI SOCIET", "si allega", "lettera"]
    
    for i,f in enumerate(e for e in os.listdir(dirr) if e[-3:] == "pdf"):
        text = ""
        inizio = -1
        index, index_fine = -1, -1
        fine = -1
        #if i == 0:
        #    continue
        if i == max:
            break
        if is_pdf and f[-3:] == 'pdf':
            text = textract.process(dirr + f).strip()
        #sents = [wd.to_utf8(g.strip()) for g in text.split("\n") if len(g.strip()) > 0]
        sents = sentences_doc(text, rep=r'<DOT>')  
        print(f)
        
        for k, frase in enumerate(sents):
            found = False
            if index == -1:
                for t in its_a_trap:
                    if wd.index_ignore_whitespaces(frase, t) > -1:
                        found = True
                        break
                if found:
                    continue

                for b in begin_statuto:
                    j = wd.index_ignore_whitespaces(frase, b)
                    if j > -1:
                        #print (frase, '\n\n')
                        inizio = k
                        index = j
                        break
                        
        if index > -1:     
            #print (sents[inizio][index:], '\n\n')
            # end_statuto_text restituisce la prima frase dopo la fine dello statuto
            res = an.end_statuto_init_text(wd.clean_string_not_compressed(text), sents[inizio][index:])
            # old func
            # res = an.end_statuto_text(wd.to_utf8(text))
            if res != '':
                res = res + '\n'
                for y, frase2 in reversed(list(enumerate(sents))):
                    if res in frase2:
                        fine = y
                        index_fine = frase2.index(res)
                        break
            
            for z, fr in enumerate(sents):
                if "RELAZIONE DI STIMA" in fr:
                    fine = z
                    index_fine = fr.index("RELAZIONE DI STIMA")
                    break
        

        if index > -1 and fine > -1:
            to_write = [wd.compress_blanks(s.strip()) for s in sents[inizio+1:fine]]
            with codecs.open(outf, 'a', encoding='utf-8') as o:
                o.write(f + '\n\n' + wd.compress_blanks(sents[inizio][index:])+'\n'+'\n'.join(to_write)
                        + '\n' + wd.compress_blanks(sents[fine][:index_fine]) + '\n\n\n\n\n')  
                
        elif index > -1 and fine == -1:
            to_write = [wd.compress_blanks(s.strip()) for s in sents[inizio+1:]]
            with codecs.open(outf, 'a', encoding='utf-8') as o:
                o.write(f + '\n\n' + wd.compress_blanks(sents[inizio][index:])+'\n'+'\n'.join(to_write) + '\n\n\n\n\n')
                
        else:
            with codecs.open(outf, 'a', encoding='utf-8') as o:
                o.write(f + '\n\n\n\n\n')

In [ ]:
# rimuove il file solo se esiste
os.path.exists('../statuti/molesti/out.txt') and os.remove('../statuti/molesti/out.txt')
extract_statuto_from_sentences('../statuti/molesti/', '../statuti/molesti/out.txt', True, -1)

In [6]:
# rimuove il file solo se esiste
os.path.exists('../statuti/test/out.txt') and os.remove('../statuti/test/out.txt')
extract_statuto_from_sentences('../statuti/test/', '../statuti/test/out.txt', True, -1)

5089268490001.pdf
4918631180001.pdf
4986499490001.pdf
4991657760001.pdf
4952252910001.pdf
5097835830001.pdf
5062352760001.pdf
4918996320001.pdf
5050824500001.pdf
5000210520001.pdf


In [ ]:
text = te.extract_text('../statuti/test/5000210520001.pdf')
lines = [l.strip() for l in text.split('\n') if len(l.strip()) > 0]
list(enumerate(lines))

In [ ]:
temp = an.matches_list(lines)
an.filter_matches_errors(temp)

In [ ]:
#print('\n\n\n'.join(s.replace('\n','< >') for s in sentences_doc(text, rep=r'<DOT>')))
#sentences_doc(text, rep=r'<DOT>')
print (text)